In [ ]:
#!pip install -r requirements.txt

# Introduction

In [ ]:
from langchain.llms import VertexAI
llm = VertexAI(temperature=0)

In [ ]:
message = "Hello my name is Sergio. I am from Barcelona and I work at Google."
msg_template = """
Given an input message, find the name, company name and location and extract them as output.
message: {message}
"""

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(msg_template)

In [ ]:
chain = prompt_template | llm
response = chain.invoke({"message": message})
print(response)

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

name_schema = ResponseSchema(name="name",
                             description="What is the name of the sender of the message?")
company_schema = ResponseSchema(name="company",
                             description="What is the name of the company of the sender of the message?")
location_schema = ResponseSchema(name="location",
                             description="What is the location of the sender of the message?")
response_schemas = [name_schema, company_schema, location_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
msg_template = """
Given an introduction message, find the name, company name and location and extract them as output.
message: {message}

{format_instructions}
"""
prompt_template = PromptTemplate.from_template(msg_template)

In [ ]:
chain = prompt_template | llm
response = chain.invoke({"message": message, "format_instructions": format_instructions})
print(response)

In [ ]:
output_dict = output_parser.parse(response)
print(output_dict)
output_dict.get("name")

# Mmeory

In [ ]:
llm("My name is Sergio")

In [ ]:
llm("What is my name?")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="My name is Sergio")

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
print(memory.buffer)

# Chains

In [ ]:
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

chain_one = LLMChain(llm=llm, prompt=prompt_template, output_key="json_values", output_parser=output_parser, verbose=True)
print(chain_one.output_keys)

In [ ]:
response = chain_one.invoke({"message": message, "format_instructions": format_instructions})
print(response.get("json_values"))

In [ ]:
prompt_2 = """
What is the main language is spoken at the location found at {json_values}?

Example: London
Result: English
"""
prompt_2_template = PromptTemplate.from_template(prompt_2)
chain_two = LLMChain(llm=llm, prompt=prompt_2_template, output_key="language", verbose=True)

In [ ]:
response_2 = chain_two.invoke({"json_values": response.get("json_values")})
print(response_2.get("language"))

In [ ]:
prompt_3 = """
Given a {json_values} write a welcome message addressed to the name, from the company that person works.
"""
prompt_3_template = PromptTemplate.from_template(prompt_3)
chain_three = LLMChain(llm=llm, prompt=prompt_3_template, output_key="welcome", verbose=True)

In [ ]:
response_3 = chain_three.invoke({"json_values": response.get("json_values")})
print(response_3.get("welcome"))

In [ ]:
prompt_4 = """
Translate {welcome} to {language}.
"""
prompt_4_template = PromptTemplate.from_template(prompt_4)
chain_four = LLMChain(llm=llm, prompt=prompt_4_template, output_key="translated", verbose=True)

In [ ]:
response_4 = chain_four.invoke({"welcome": response_3.get("welcome"), "language": response_2.get("language")})
print(response_4.get("translated"))

In [ ]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["message", "format_instructions"],
    output_variables=["json_values", "language","welcome", "translated"],
    verbose=True
)

In [ ]:
result = overall_chain.invoke({"message": message, "format_instructions": format_instructions})

In [ ]:
print(result.get("translated"))

# Agents

In [ ]:
from langchain.agents import tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
import uuid

In [ ]:
@tool
def validate_client(client: str = None) -> bool:
    """Always validate the client first. The company name must be given by the user. When a user claims to work at a company, checks if a client is in a list of accepted clients. If no user or not validated, stop. The user is validated if the return is True, it is not validated if it is False"""
    clients = ["ACME", "Google"]
    return client in clients

@tool
def get_code(client: str) -> str:
    """After validating that the client is accepted, generate an access code and return it."""
    code = client.upper() + '#' + str(uuid.uuid4())
    return code

In [ ]:
agent = initialize_agent(
    [validate_client, get_code], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
agent_result = agent("Is Google an accepted client?")

In [ ]:
agent_result = agent("I work at Google, give me a code")

In [ ]:
agent_result = agent("I work at Giigles, give a code")
print(agent_result)

In [ ]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four, agent],
    input_variables=["message", "format_instructions", "input"],
    output_variables=["json_values", "language","welcome", "translated", "output"],
    verbose=True
)

In [ ]:
overall_chain.invoke({"message": message, "format_instructions": format_instructions, 'input': "Google"})